In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
raw_dataset = pd.read_csv('../raw_data/global-shark-attack.csv', sep=';')

In [3]:
# Dataset overview
raw_dataset.info()
raw_dataset.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6890 entries, 0 to 6889
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    6587 non-null   object 
 1   Year                    6758 non-null   float64
 2   Type                    6871 non-null   object 
 3   Country                 6839 non-null   object 
 4   Area                    6409 non-null   object 
 5   Location                6325 non-null   object 
 6   Activity                6304 non-null   object 
 7   Name                    6670 non-null   object 
 8   Sex                     6318 non-null   object 
 9   Age                     3903 non-null   object 
 10  Injury                  6853 non-null   object 
 11  Fatal (Y/N)             6890 non-null   object 
 12  Time                    3372 non-null   object 
 13  Species                 3772 non-null   object 
 14  Investigator or Source  6869 non-null   

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number,Case Number.1,original order
0,2023-07-04,2023.0,Unprovoked,USA,New York,"Cherry Grove, Suffolk County",Standing,female,F,NaN,...,N,16h25,4' shark,"S.S. Curatolo- Wagemann, GSAFF",NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2023.0,Unprovoked,USA,Florida,"Outer Mole, Fort Zachrry Tatlor Historic Park,...",Swimming,Nicolae Cioban,M,34,...,N,14h00,NaN,"Miami Herald, 1/14/2023",NaN,NaN,NaN,NaN,NaN,NaN
2,2022-07-19,2022.0,Questionable,USA,Florida,"Cape Canaveral, Brevard County",NaN,NaN,NaN,NaN,...,N,10jh45,Shark involvement not confirmed,"WESH, 7/20/2022",2022.07.19.b-Canaveral.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2022.07.19.b,2022.07.19.b,6788.0
3,2022-02-05,2022.0,Unprovoked,AUSTRALIA,Western Australia,"Beds, Wylie Bay, Esperance",Floating in inflatable pool ring,Jacquelin Morle,F,20.0,...,N,11h15,White shark,"K. McMurray, TrackingSharks.com",2022.02.05-Morle.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2022.02.05,2022.02.05,6734.0
4,2022-01-09,2022.0,Unprovoked,NEW ZEALAND,South Island,"Friendly Bay, Oamaru",Swimming,Alvira Repia-King,F,13.0,...,N,16h30,Sevengill shark,"Otago Daily Times, 1/9/2022",2022.01.09-Repia-King.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2022.01.09,2022.01.09,6730.0


In [4]:
print(raw_dataset.columns.tolist())

['Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href', 'Case Number', 'Case Number.1', 'original order']


In [5]:
# drop unnecessary columns
raw_dataset = raw_dataset.rename(columns={'Sex ': 'Sex'})
columns_to_keep = ['Date', 'Country', 'Area', 'Location', 'Sex', 'Fatal (Y/N)']
raw_dataset = raw_dataset[columns_to_keep]
raw_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6890 entries, 0 to 6889
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Date         6587 non-null   object
 1   Country      6839 non-null   object
 2   Area         6409 non-null   object
 3   Location     6325 non-null   object
 4   Sex          6318 non-null   object
 5   Fatal (Y/N)  6890 non-null   object
dtypes: object(6)
memory usage: 323.1+ KB


In [6]:
# check for empty
raw_dataset.isna().sum()

Date           303
Country         51
Area           481
Location       565
Sex            572
Fatal (Y/N)      0
dtype: int64

In [7]:
raw_dataset = raw_dataset.dropna(subset=['Date','Sex'])
raw_dataset.isna().sum()

Date             0
Country         41
Area           375
Location       443
Sex              0
Fatal (Y/N)      0
dtype: int64

In [8]:
# find and fix/remove rows with wrong date format
date_pattern = r'^\d{4}-\d{2}-\d{2}$'
valid_format = raw_dataset['Date'].str.match(date_pattern, na=False)
non_standard_dates = raw_dataset[~valid_format & ~raw_dataset['Date'].isna()]
print("Row(s) to fix:")
print(non_standard_dates['Date'])

Row(s) to fix:
4713    202-07-10
4889    144-07-22
Name: Date, dtype: object


In [9]:
date_count_before = raw_dataset['Date'].count()
raw_dataset = raw_dataset[valid_format]
date_count_after = raw_dataset['Date'].count()
print("Row(s) removed: ", date_count_before-date_count_after)

Row(s) removed:  2


In [10]:
# remove dates < '1900-01-01'
print("Total count: ",raw_dataset['Date'].count())
pre_1900_count = (raw_dataset['Date'] < '1900-01-01').sum()
print("Before 1900: ",pre_1900_count)
after_1900_count = (raw_dataset['Date'] >= '1900-01-01').sum()
print("After 1900: ",after_1900_count)

Total count:  6045
Before 1900:  513
After 1900:  5532


In [11]:
raw_dataset = raw_dataset[(raw_dataset['Date'] >= '1900-01-01')]
print("New total count: ",raw_dataset['Date'].count())

New total count:  5532


In [12]:
# fix Fatal column
raw_dataset = raw_dataset.rename(columns={'Fatal (Y/N)': 'Fatal'})
unique_fatal_values = raw_dataset['Fatal'].value_counts()
print(unique_fatal_values)

Fatal
N          4113
Y           974
UNKNOWN     438
F             3
Y x 2         1
2017.0        1
n             1
Nq            1
Name: count, dtype: int64


In [13]:
# conver to True, False, or NaN
raw_dataset['Fatal'] = raw_dataset['Fatal'].map({ 'Y': True, 'y': True, 'Y x 2': True,\
                                                 'N': False, 'n': False, 'Nq': False,\
                                                 'F': False}).fillna(value=np.nan)  # all others (UNKNOWN, 2017.0) become NaN

In [14]:
print(raw_dataset['Fatal'].value_counts(dropna=False))

Fatal
False    4118
True      975
NaN       439
Name: count, dtype: int64


In [15]:
# converting everything to string first to handle mixed data types
raw_dataset['Country'] = raw_dataset['Country'].astype(str)
# identify unique values and if need fix
unique_countries = raw_dataset['Country'].unique()
print("Number of unique countries: ", len(unique_countries))
print("\nUnique country values:")
for country in sorted(unique_countries):
    print(f"- {country}")

Number of unique countries:  174

Unique country values:
- ADMIRALTY ISLANDS
- AMERICAN SAMOA
- ANDAMAN / NICOBAR ISLANDAS
- ANTIGUA
- ARGENTINA
- ARUBA
- ATLANTIC OCEAN
- AUSTRALIA
- AZORES
- Australia
- BAHAMAS
- BARBADOS
- BAY OF BENGAL
- BELIZE
- BERMUDA
- BRAZIL
- BRITISH ISLES
- BRITISH VIRGIN ISLANDS
- BRITISH WEST INDIES
- British Overseas Territory
- CANADA
- CAPE VERDE
- CARIBBEAN SEA
- CAYMAN ISLANDS
- CENTRAL PACIFIC
- CHILE
- CHINA
- COLOMBIA
- COLUMBIA
- COSTA RICA
- CROATIA
- CUBA
- CURACAO
- CYPRUS
- DIEGO GARCIA
- DOMINICAN REPUBLIC
- ECUADOR
- EGYPT
- EGYPT / ISRAEL
- EL SALVADOR
- ENGLAND
- FALKLAND ISLANDS
- FEDERATED STATES OF MICRONESIA
- FIJI
- FRANCE
- FRENCH POLYNESIA
- Fiji
- GABON
- GRAND CAYMAN
- GREECE
- GRENADA
- GUAM
- GUATEMALA
- GUYANA
- HAITI
- HONDURAS
- HONG KONG
- ICELAND
- INDIA
- INDIAN OCEAN
- INDONESIA
- IRAN
- IRAN / IRAQ
- IRAQ
- IRELAND
- ISRAEL
- ITALY
- ITALY / CROATIA
- JAMAICA
- JAPAN
- JOHNSTON ISLAND
- JORDAN
- KENYA
- KIRIBATI
- KUWAIT

In [19]:
# NEED TO CLEAN 'Country' column
# fist uppercase all
raw_dataset['Country'] = raw_dataset['Country'].str.upper()
# clean up specific country names by replacement names
country_mapping = {
    'COLUMBIA': 'COLOMBIA',
    'GRAND CAYMAN': 'BRITISH OVERSEAS TERRITORY',
    'MALDIVE ISLANDS': 'MALDIVES',
    'REUNION': 'REUNION ISLAND',
    'ST. MAARTIN': 'ST MARTIN',
    'ST. MARTIN': 'ST MARTIN',
    'SUDAN?': 'SUDAN',
    'TURKS AND CAICOS': 'TURKS & CAICOS',
    'UNITED ARAB EMIRATES (UAE)': 'UNITED ARAB EMIRATES'
}
raw_dataset['Country'] = raw_dataset['Country'].replace(country_mapping)
# lastly remove NAN strings (missed earlier)
raw_dataset = raw_dataset[raw_dataset['Country'] != 'NAN']
# identify unique values and if need fix
unique_countries = raw_dataset['Country'].unique()
print("Number of unique countries: ", len(unique_countries))
print("\nUnique country values:")
for country in sorted(unique_countries):
    print(f"- {country}")

Number of unique countries:  156

Unique country values:
- ADMIRALTY ISLANDS
- AMERICAN SAMOA
- ANDAMAN / NICOBAR ISLANDAS
- ANTIGUA
- ARGENTINA
- ARUBA
- ATLANTIC OCEAN
- AUSTRALIA
- AZORES
- BAHAMAS
- BARBADOS
- BAY OF BENGAL
- BELIZE
- BERMUDA
- BRAZIL
- BRITISH ISLES
- BRITISH OVERSEAS TERRITORY
- BRITISH VIRGIN ISLANDS
- BRITISH WEST INDIES
- CANADA
- CAPE VERDE
- CARIBBEAN SEA
- CAYMAN ISLANDS
- CENTRAL PACIFIC
- CHILE
- CHINA
- COLOMBIA
- COSTA RICA
- CROATIA
- CUBA
- CURACAO
- CYPRUS
- DIEGO GARCIA
- DOMINICAN REPUBLIC
- ECUADOR
- EGYPT
- EGYPT / ISRAEL
- EL SALVADOR
- ENGLAND
- FALKLAND ISLANDS
- FEDERATED STATES OF MICRONESIA
- FIJI
- FRANCE
- FRENCH POLYNESIA
- GABON
- GREECE
- GRENADA
- GUAM
- GUATEMALA
- GUYANA
- HAITI
- HONDURAS
- HONG KONG
- ICELAND
- INDIA
- INDIAN OCEAN
- INDONESIA
- IRAN
- IRAN / IRAQ
- IRAQ
- IRELAND
- ISRAEL
- ITALY
- ITALY / CROATIA
- JAMAICA
- JAPAN
- JOHNSTON ISLAND
- JORDAN
- KENYA
- KIRIBATI
- KUWAIT
- LIBERIA
- LIBYA
- MADAGASCAR
- MALAYSIA
- 

In [21]:
raw_dataset.to_csv('../processed_data/clean_shark_attacks_data.csv', index=False)
raw_dataset.to_json('../processed_data/clean_shark_attacks_data.json', orient='records')
# since React cannot access files outside the src, add JSON straight to ../src/data folder
raw_dataset.to_json('../../shark-attacks-viz-app/src/data/clean_shark_attacks_data.json', orient='records')